In [1]:
import gymnasium as gym
from gymnasium import spaces
from collections import defaultdict
import numpy as np

class NegativeHoleRewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)

    def step(self, action):
        observation, reward, terminated, truncated, info = self.env.step(action)
        if reward == 0 and terminated:
            reward = -1
        
        return observation, reward, terminated, truncated, info

env = gym.make('FrozenLake-v1', desc=["SFFF", "FHFH", "FFFH", "HFFG"], is_slippery=False)
wrapped_env = NegativeHoleRewardWrapper(env)
observation, info = wrapped_env.reset(seed=42)